In [20]:
### import necessary libraries
import ee
import geemap
import sklearn
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [21]:
ee.Authenticate()
ee.Initialize(project="omdena-italy")

In [26]:
myList = ee.List.sequence(1, 10)
print(myList.getInfo())

mean = myList.reduce(ee.Reducer.mean())
print(mean.getInfo())

# we want to know the min and max of the list
minMax = myList.reduce(ee.Reducer.minMax())
print(minMax.getInfo())

geometry = ee.Geometry.Polygon([[82.60642647743225, 27.16350437805251],
  [82.60984897613525, 27.1618529901377],
  [82.61088967323303, 27.163695288375266],
  [82.60757446289062, 27.16517483230927]])

# create a collection using copernicus s2_harmonized data
s2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")

Map = geemap.Map()
Map.centerObject(geometry, 15)
Map.addLayer(geometry, vis_params={'color': 'red'}, name='Polygon Layer')

# Apply a reducer on a image collection using a variable called filtered
filtered = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
            .filter(ee.Filter.bounds(geometry)) \
            .select('B.*')

print(filtered.getInfo())

# Map.addLayer(filtered.median(), vis_params={'bands': ['B4', 'B3', 'B2'], 'max': 2000}, name='Filtered Image')

Map

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
5.5
{'max': 10, 'min': 1}
{'type': 'ImageCollection', 'bands': [], 'id': 'COPERNICUS/S2_HARMONIZED', 'version': 1707679846061350.0, 'properties': {'date_range': [1435017600000, 1647993600000], 'period': 0, 'system:visualization_0_min': '0.0', 'type_name': 'ImageCollection', 'keywords': ['copernicus', 'esa', 'eu', 'msi', 'radiance', 'sentinel'], 'system:visualization_0_bands': 'B4,B3,B2', 'thumb': 'https://mw1.google.com/ges/dd/images/sentinel2_thumb.png', 'description': '<p>Sentinel-2 is a wide-swath, high-resolution, multi-spectral\nimaging mission supporting Copernicus Land Monitoring studies,\nincluding the monitoring of vegetation, soil and water cover,\nas well as observation of inland waterways and coastal areas.</p><p>The Sentinel-2 data contain 13 UINT16 spectral bands representing\nTOA reflectance scaled by 10000. See the <a href="https://sentinel.esa.int/documents/247904/685211/Sentinel-2_User_Handbook">Sentinel-2 User Handbook</a>\nfor details

Map(center=[27.163542632604905, 82.6086955256044], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
# We want to create a median composite
median = filtered.median()
composite = filtered.reduce(ee.Reducer.median())

In [31]:
image = ee.Image('COPERNICUS/S2_HARMONIZED/20190223T050811_20190223T051829_T44RPR')
rgbVis = { 'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000 }
Map.addLayer(image, rgbVis, 'Image')
Map.addLayer(geometry, vis_params={'color': 'red'},name='Farm')
Map

Map(bottom=14145285.0, center=[27.162180227002533, 82.61412978172304], controls=(WidgetControl(options=['posit…

In [32]:
stats = image.reduceRegion(
    reducer=ee.Reducer.mean(), 
    geometry=geometry, 
    scale=10
    )
print(stats.getInfo())

{'B1': 1232.6209971910116, 'B10': 8.125847378277156, 'B11': 1180.017940074907, 'B12': 529.8835955056179, 'B2': 953.1895177902626, 'B3': 846.8504447565547, 'B4': 511.805571161049, 'B5': 829.4386235955062, 'B6': 2691.8323501872683, 'B7': 3590.9208988764044, 'B8': 3473.0389606741587, 'B8A': 3792.154854868916, 'B9': 1115.446343632959, 'MSK_CLASSI_CIRRUS': None, 'MSK_CLASSI_OPAQUE': None, 'MSK_CLASSI_SNOW_ICE': None, 'QA10': 0, 'QA20': 0, 'QA60': 0}


In [33]:
ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
print(ndvi.getInfo())

{'type': 'Image', 'bands': [{'id': 'NDVI', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -1, 'max': 1}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32644', 'crs_transform': [10, 0, 600000, 0, -10, 3100020]}], 'properties': {'system:footprint': {'type': 'LinearRing', 'coordinates': [[83.13351474963679, 28.00895984084098], [83.13349797576093, 28.008962915182305], [82.01723535319307, 28.02176044018027], [82.01718803069325, 28.021724232030717], [82.01713272404155, 28.021691641656165], [82.01253463821232, 27.52622468538399], [82.00805270859865, 27.03074214206233], [82.00809338283378, 27.03070030199569], [82.00812998130793, 27.030651540580884], [83.09571654580809, 27.01866075174234], [83.09575346964355, 27.01868883156147], [83.09580079106969, 27.018698835050674], [83.09581266235635, 27.018718717479278], [83.09665666439128, 27.020694604011172], [83.09836395664186, 27.025727628611115], [83.10266342779563, 27.040105530612905], [83.11216998829235, 27.074080363111968], [83.115927

In [35]:
# Compute the average NDVI for the farm from the image. hint: use the reduceRegion function
ndvi_stats = ndvi.reduceRegion(
    reducer=ee.Reducer.mean(), 
    geometry=geometry, 
    scale=10
    )
print(ndvi_stats.getInfo())
Map.addLayer(ndvi, {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'NDVI')
Map

{'NDVI': 0.7404287597896536}


Map(bottom=3536576.0, center=[27.16104429036319, 82.6119089126587], controls=(WidgetControl(options=['position…

In [36]:
#Create a Map to see ndvi
ndviVis = {
  'min': 0.0,
  'max': 1.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ]
}
Map.addLayer(ndvi, ndviVis, 'NDVI')
Map

Map(bottom=3536576.0, center=[27.16104429036319, 82.6119089126587], controls=(WidgetControl(options=['position…